In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Importando todos os arquivos csv, distintos por mês

mes_1 = pd.read_csv(r"../dados de viagens ciclísticas dos últimos 12 meses/202401-divvy-tripdata.csv")
mes_2 = pd.read_csv(r"../dados de viagens ciclísticas dos últimos 12 meses/202402-divvy-tripdata.csv")
mes_3 =  pd.read_csv("../dados de viagens ciclísticas dos últimos 12 meses/202403-divvy-tripdata.csv")
mes_4 = pd.read_csv(r"../dados de viagens ciclísticas dos últimos 12 meses/202404-divvy-tripdata.csv")
mes_5 = pd.read_csv(r"../dados de viagens ciclísticas dos últimos 12 meses/202405-divvy-tripdata.csv")
mes_6 = pd.read_csv(r"../dados de viagens ciclísticas dos últimos 12 meses/202406-divvy-tripdata.csv")
mes_7 =  pd.read_csv(r"../dados de viagens ciclísticas dos últimos 12 meses/202407-divvy-tripdata.csv")
mes_8 = pd.read_csv(r"../dados de viagens ciclísticas dos últimos 12 meses/202408-divvy-tripdata.csv")
mes_9 =  pd.read_csv(r"../dados de viagens ciclísticas dos últimos 12 meses/202409-divvy-tripdata.csv")
mes_10 = pd.read_csv(r"../dados de viagens ciclísticas dos últimos 12 meses/202410-divvy-tripdata.csv")
mes_11 = pd.read_csv(r"../dados de viagens ciclísticas dos últimos 12 meses/202411-divvy-tripdata.csv")
mes_12 =  pd.read_csv(r"../dados de viagens ciclísticas dos últimos 12 meses/202412-divvy-tripdata.csv")

In [ ]:
# Convertendo os arquivos csv em DataFrame

mes_1_df = pd.DataFrame(mes_1)
mes_2_df = pd.DataFrame(mes_2)
mes_3_df = pd.DataFrame(mes_3)
mes_4_df = pd.DataFrame(mes_4)
mes_5_df = pd.DataFrame(mes_5)
mes_6_df = pd.DataFrame(mes_6)
mes_7_df = pd.DataFrame(mes_7)
mes_8_df = pd.DataFrame(mes_8)
mes_9_df = pd.DataFrame(mes_9)
mes_10_df = pd.DataFrame(mes_10)
mes_11_df = pd.DataFrame(mes_11)
mes_12_df = pd.DataFrame(mes_12)

In [ ]:
# Concatenando os dados do dataset, reinicializando os índices e removendo duplicatas 

ano = pd.concat([mes_1_df, mes_2_df, mes_3_df, mes_4_df, mes_5_df, mes_6_df, mes_7_df, mes_8_df, mes_9_df, mes_10_df, mes_11_df, mes_12_df], ignore_index=True)
ano = ano.drop_duplicates()

In [ ]:
# Propriedades do DataFrame
print(f"Quantidade de linhas:\n{ano.shape[0]}"); print("-" * 35)
print(f"Quantidade de colunas:\n{ano.shape[1]}"); print("-" * 35)
print(f"Tipos de dados:\n{ano.dtypes}"); print("-" * 35)
print(f"Tamanho do arquivo:\n{ano.size}"); print("-" * 35)

In [ ]:
# Verificando inconsistência nos dados e valores nulos
valores_nulos = ano.isnull().sum()
print(f"Checando valores nulos:\n{valores_nulos}");print("-" * 35)

In [ ]:
# Removendo valores nulos
ano = ano.dropna()

In [ ]:
# Convertendo as colunas 'started_at e 'ended_at' de objetos para datetime

ano['started_at'] = pd.to_datetime(ano['started_at'], format='mixed')
ano['ended_at'] = pd.to_datetime(ano['ended_at'], format='mixed')

In [ ]:
# Criando a coluna 'ride_length' que mostra o resultado do ínicio de viagem de bicicleta até o término

ano['ride_length'] = ano['ended_at'] - ano['started_at']

ano.info()
ano['ride_length'].head()

In [ ]:
# Criando uma coluna chamada 'day_of_week' com o ínicio de 

ano['day_of_week'] = ano['started_at'].dt.day_name()
ano['day_of_week']

In [ ]:
# Checando inconsistência nos dados da coluna 'ride-length'

teste = ano.sort_values(by='ride_length', ascending=True)

In [ ]:
# Amostra das 10 primeiras mostram que a coluna 'ride_length' há passeios com tempo negativo 

teste['ride_length'].head(10)

In [ ]:
# Filtragem para os dias menor que 0

filtragem = ano['ride_length'].dt.days < 0 

In [ ]:
# Removendo dias negativos

ano.drop(ano[filtragem].index, inplace=True)

ano['ride_length']

In [ ]:
# Removendo passeios com menos de 60 segundos

filtragem_segundo = ano['ride_length'].dt.seconds <= 60
ano.drop(ano[filtragem_segundo].index, inplace=True)

ano['ride_length'].sort_values(ascending=True).head(10)

In [ ]:
# Media do tempo de bicicleta por membro
# Máxima do do tempo de passeio 
# Moda dos dias da semana 

media_tempo_bicicleta = ano["ride_length"].mean()
tempo_maximo_viagem = ano['ride_length'].max()
moda_dias_semana = ano['day_of_week'].mode()

print(f'''
Media do tempo de bicicleta por membro:\t{media_tempo_bicicleta}\n
Máxima do do tempo de passeio:{tempo_maximo_viagem}\n
Moda dos dias da semana:{moda_dias_semana}''')

In [ ]:
# Agrupando a média e total de viagens de bicicleta por TIPO membro

media_por_membro = ano.groupby(['member_casual'])['ride_length'].mean()
total_membro = ano['member_casual'].value_counts()

# Transformando a Serie total_membro em DataFrame e resetando o índices
total_membrodf = pd.DataFrame(total_membro).reset_index()
total_membrodf

# Função para mostrar os valores totais de cada variável
def formatar_valores(pct, valores):
    total = sum(valores)
    valor = int(round(pct * total / 100.0))
    return f'{valor}'

# Plotando o valor total por tipo de membro
plt.pie(total_membro,autopct=lambda pct: formatar_valores(pct, total_membro))
plt.title('Total membros por categoria')
plt.legend(total_membrodf, labels=total_membrodf['member_casual'])

print(f''' 
Média por TIPO de membro:{media_por_membro}
\n
Totais usuários por TIPO de membro:{total_membro}
''')

In [ ]:
# Agrupando a média de viagens feitas por dia semana

media_por_semana = ano.groupby(['day_of_week'])['ride_length'].mean() 
print(f'A média de viagens feitas por dia semana\n{media_por_semana}')

media_por_semanadf = pd.DataFrame(media_por_semana).reset_index()
media_por_semanadf.plot(kind='bar', x='day_of_week', y='ride_length', legend=False)



plt.title('Média de tempo por passeios na semana')
plt.ylabel('Minutos')
plt.xlabel('Dias da Semana')
plt.show()

In [ ]:
# Agrupando o total de viagens por dia da semana

contagem_viagens_semana = ano.groupby(['day_of_week'])['ride_length'].count(); print(contagem_viagens_semana)

contagem_viagens_ordenadas = contagem_viagens_semana.sort_values()

contagem_viagens_ordenadas.plot(kind='bar')
plt.title("Quantidade total de passeios por dia da semana")
plt.xlabel("Dias da Semana")

In [ ]:
# Porcentagem Ciclistas Casuais vs Membros Cyclistic

total_membro.plot(title='Porcentagem - Ciclista Casuais vs Membros Cyclistc', 
                        kind='pie',
                        autopct='%1.1f%%',
                        shadow=True)

In [ ]:
# Quantidade total por tipo de bicicleta

agrupado_total = ano.groupby('rideable_type')['ride_id'].count().reset_index()
agrupado_totaldf = pd.DataFrame(agrupado_total)

plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x/10000:.0f}Mil'))  # Converter para milhares


plt.bar(agrupado_totaldf['rideable_type'], agrupado_totaldf['ride_id'])
plt.title('Quantidade total por tipo de bicicleta')
plt.ylabel('Valores (em milhares)')

In [ ]:
# Separando o total de cada tipo de membro por dia da semana
ano_grupos = ano[['rideable_type','member_casual', 'day_of_week']].value_counts()
anos_gruposdf = pd.DataFrame(ano_grupos)
anos_gruposdf.groupby('member_casual')
anos_gruposdf_sorted = anos_gruposdf.sort_values(by=['member_casual'], ascending=False).reset_index()
anos_gruposdf_sorted_1 = anos_gruposdf_sorted.sort_values(by=['member_casual'])

anos_gruposdf_sorted_1

In [ ]:
# Criando uma coluna 'mes' correspondente com cada viagem por usuário
meses_ano = {
    1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 4: 'Abril',
    5: 'Maio', 6: 'Junho', 7: 'Julho', 8: 'Agosto',
    9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'
}

ano['mes'] = ano['started_at'].dt.month.map(meses_ano)

ano.head(25)

In [ ]:
#1.  Indexando os meses em ordem
#2. Criando o gráfico da quantidade de passeios ao longo do ano.


ano_mes_df = ano['mes'].value_counts().reset_index()
novo_indice = np.array([7,6,8,5,9,4,3,10,2,1,11,0])
ano_mes_df.index = novo_indice
ano_mes_df_1 = ano_mes_df.sort_index()
ano_mes_df_1.plot(kind='bar', x='mes', y='count', legend=False)
plt.xlabel('Meses')
plt.title('Quantidade de passeios ao longo do ano')
plt.show()